In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [68]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

In [9]:
trainning = pd.read_csv(r"C:\Users\user\Documents\Projects ML\Naive Bayes to predict loan approval\train.csv")
test = pd.read_csv(r"C:\Users\user\Documents\Projects ML\Naive Bayes to predict loan approval\test.csv")

In [10]:
train = trainning.copy()

In [11]:
train.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [13]:
train.shape

(614, 13)

In [15]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             601 non-null    object 
 2   Married            611 non-null    object 
 3   Dependents         599 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      582 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         592 non-null    float64
 9   Loan_Amount_Term   600 non-null    float64
 10  Credit_History     564 non-null    float64
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 62.5+ KB


In [22]:
#replace Y and N in loan status with 0 and 1
train['Loan_Status'].replace('N',0,inplace=True)
train['Loan_Status'].replace('Y',1,inplace=True)

In [23]:
train.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,1
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,0
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,1
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,1
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,1


In [19]:
train.isnull().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

# Data Preprocessing

In [24]:
# Replace category wise missing values
train['Gender'] = train['Gender'].replace(np.nan,train['Gender'].mode()[0],regex=True)

train['Married'] = train['Married'].replace(np.nan,train['Married'].mode()[0],regex=True)

train['Dependents'] = train['Dependents'].replace(np.nan,train['Dependents'].mode()[0],regex=True)

train['Self_Employed'] = train['Self_Employed'].replace(np.nan,train['Self_Employed'].mode()[0],regex=True)

train['Credit_History'] = train['Credit_History'].replace(np.nan,train['Credit_History'].mode()[0],regex=True)

In [25]:
train.isnull().sum()

Loan_ID               0
Gender                0
Married               0
Dependents            0
Education             0
Self_Employed         0
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History        0
Property_Area         0
Loan_Status           0
dtype: int64

In [26]:
# Now replace continuous missing values with the median for train
median_loanAmount = train['LoanAmount'].median()
median_loanAmount

128.0

In [28]:
train['LoanAmount'] = train['LoanAmount'].replace(np.nan,median_loanAmount,regex=True,inplace=True)
train['LoanAmount'].fillna(median_loanAmount,inplace=True)

In [33]:
train['Loan_Amount_Term'].fillna(360,inplace = True)

In [34]:
train.isnull().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

# Feature Engineering


In [36]:
# Calculate the total income
train['Total_Income'] = train['ApplicantIncome'] + train['CoapplicantIncome']

In [37]:
#Creating the EMI Feature
train['EMI'] = train['LoanAmount'] / train['Loan_Amount_Term']

In [38]:
#Creating new Balance Income variable
train['Balance_Income'] = train['Total_Income'] - (train['EMI']*1000) #Multiply with 1000 to make the units equal

In [39]:
train.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,Total_Income,EMI,Balance_Income
0,LP001002,Male,No,0,Graduate,No,5849,0.0,128.0,360,1.0,Urban,1,5849.0,0.355556,5493.444444
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360,1.0,Rural,0,6091.0,0.355556,5735.444444
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,128.0,360,1.0,Urban,1,3000.0,0.355556,2644.444444
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,128.0,360,1.0,Urban,1,4941.0,0.355556,4585.444444
4,LP001008,Male,No,0,Graduate,No,6000,0.0,128.0,360,1.0,Urban,1,6000.0,0.355556,5644.444444


In [40]:
#Drop loan_id
train = train.drop(['Loan_ID','ApplicantIncome','CoapplicantIncome','Loan_Amount_Term'],axis=1)

In [41]:
# Adding dummies to the dataset
train = pd.get_dummies(train)

In [42]:
train.head()

,LoanAmount,Credit_History,Loan_Status,Total_Income,EMI,Balance_Income,Gender_Female,Gender_Male,Married_No,Married_Yes,...,Dependents_1,Dependents_2,Dependents_3+,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban
0,128.0,1.0,1,5849.0,0.355556,5493.444444,0,1,1,0,...,0,0,0,1,0,1,0,0,0,1
1,128.0,1.0,0,6091.0,0.355556,5735.444444,0,1,0,1,...,1,0,0,1,0,1,0,1,0,0
2,128.0,1.0,1,3000.0,0.355556,2644.444444,0,1,0,1,...,0,0,0,1,0,0,1,0,0,1
3,128.0,1.0,1,4941.0,0.355556,4585.444444,0,1,0,1,...,0,0,0,0,1,1,0,0,0,1
4,128.0,1.0,1,6000.0,0.355556,5644.444444,0,1,1,0,...,0,0,0,1,0,1,0,0,0,1


In [46]:
X = train.drop('Loan_Status',1)
Y = train.Loan_Status

In [48]:
X.shape

(614, 20)

In [49]:
Y.shape

(614,)

# Split data into test and train

In [62]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.3,random_state=42)

In [52]:
from sklearn.naive_bayes import GaussianNB

In [53]:
nb = GaussianNB()
nb.fit(X_train,Y_train)

GaussianNB()

In [54]:
Y_pred = nb.predict(X_test)

In [64]:
# Accuracy 
score = nb.score(X_test,Y_test)
print(score)

0.7837837837837838


In [66]:
cm  = confusion_matrix(Y_test,Y_pred)
cm

array([[ 14,  51],
       [ 15, 105]], dtype=int64)

# With K fold Cross - Validation with 10 folds

In [69]:
accuracy = cross_val_score(nb,X,Y,scoring='accuracy',cv=10)
print(accuracy)

[0.80645161 0.82258065 0.70967742 0.77419355 0.78688525 0.80327869
 0.85245902 0.83606557 0.75409836 0.86885246]


In [73]:
# Get the mean of folds
a = accuracy.mean()*100
a

80.14542570068747